<a href="https://colab.research.google.com/github/Capou/FishClassifier/blob/master/fish_classifier_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#!pip install fastbook

     |████████████████████████████████| 727kB 3.9MB/s 
     |████████████████████████████████| 51kB 6.0MB/s 
     |████████████████████████████████| 1.2MB 26.5MB/s 
     |████████████████████████████████| 194kB 33.2MB/s 
     |████████████████████████████████| 61kB 7.1MB/s 
     |████████████████████████████████| 61kB 6.7MB/s 
  Found existing installation: fastai 1.0.61
    Uninstalling fastai-1.0.61:
      Successfully uninstalled fastai-1.0.61


In [4]:
import pandas 
import fastbook
import voila

In [5]:
from fastai.vision.all import *
from fastai.vision.widgets import *
from pandas import *
from fastbook import * 

In [7]:
#from google.colab import drive
#drive.mount('/content/drive')

Mounted at /content/drive


# The Amazing Fish Classifier!

You are not sure about fish species flipping around infront of you. An answer is needed *fast* to avoid unnecessary stress for the animal. Additionally you want to keep your anglers license and stay within the governmental regulazions. No worries, I got you. Just take a picture of the fish and upload it to receive an answer. Fish-information only available in German.

(Supported species: abramis brama, ameiurus nebulosus, cobitis taenia, cyprinus carpio, esox lucius, gymnocephalus cernua, salmo trutta) 

----

In [ ]:
path = Path()
#dfs = pandas.read_json('/content/drive/MyDrive/pkl/fish-information.json')
dfs = pandas.read_json(path/'fish-information.json')
dfs

In [10]:
path = Path()
learn_inf = load_learner(path/'export-res34-047.pkl', cpu=True)
#learn_inf = load_learner('/content/drive/MyDrive/pkl/export-res34-047_final.pkl', cpu=True)
df = pandas.read_json(path/'fish-information.json')
btn_upload = widgets.FileUpload()
out_pl = widgets.Output()
out_pl_df = widgets.Output()
lbl_pred = widgets.Label()

In [11]:
def create_df(): 
    fish = {'Familie': ['Leuciscidae', 'Icaluridae', 'Cobitidae', 'Cyprinidae', 'Esocidae', 'Percidae', 'Salmonidae'],
        'Spezies': ['abramis_brama', 'ameiurus_nebulosus', 'cobitis_taenia', 'cyprinus_carpio', 'esox_lucius', 'gymnocephalus_cernua', 'salmo_trutta'],
        'Name': ['Brasse', 'Katzenwels', 'Dorngrundel', 'Karpfen', 'Hecht', 'Kaulbarsch', 'Forelle'],
        'Koerperform':['Hochrueckig', 
                       'bauch abgeplattet', 
                       'Spindel-/Torpedoform', 
                       'langgestreckt flachrueckig, walzenfoerming', 
                       'Pfeilfoermig', 
                       'seitlich abgeflacht, leicht hochrueckig', 
                       'spindel-, torpedofoermig'],
        'Mund': ['Ruesselmaul', 
                 'endständig', 
                 'unterstaendig', 
                 'endstaendig, vorstuelpbar, ruesselmaul', 
                 'oberstaendig, entenschnabelform, weite Maulspalte', 
                 'endstaendig', 
                 'endstaendig, Maulspalte reicht bis hinter die Augen, Leichhaken'],
        'Schuppen': ['realtiv groß', 
                     'keine', 
                     'sehr klein und duenn', 
                     'voll beschuppt', 
                     'klein, voll beschuppt', 
                     'klein', 
                     'kleine Rundschuppen'],
        'Farbe': ['alle Flossen dunkelgrau', 
                  'variabel', 
                  'Koerper mit laengsreihen dunkler Punkte', 
                  'Ruecken: bronze-gelb; Schwanz: roetlich', 
                  'Flossen mit dunklen Flecken, Seiten mit dunklen Querbinden', 
                  'Garu- und Gruentoene, dunkle Flecken auf Flossen', 
                  'Flanken mit schwarzen Flecken und Punkten'], 
        'Rueckenflosse': ['beginn hinter Bauchflossenansatz', 
                          'beginnt vor Bauchflossenansatz', 
                          'auffallend hoch, beginnt vor Bauchflossenansatz', 
                          '-', 
                          'weit nach hinten versetzt, Hinterrand stark gerundet',
                          'zwei, miteinander verbunden',
                          'beginnt vor Bauchlossenansatz'], 
        'Schwanzflosse': ['tief gegabelt, unsymmetrisch',
                          'gleichmäßig - gerundet',
                          'leicht gerundet',
                          '-',
                          'gegabelt',
                          'gleichmaeßig',
                          'bei erwachsenen Fisch am Hinterrand fast gerade'],
        'Afterflosse':['sehr lang, konkav',
                       'groß, konvex',
                       'konvex',
                       '-',
                       'konvex',
                       'Flossenhaut zwischen vorderen Stachelstrahl wenig eingebuchtet',
                       '-'], 
        'Brust/Bauchflosse':['reichen bis Bauchflossenansatz',
                       '-', 
                       'Bauchflosse bauchstaendig',
                       '-',
                       'Bauchflosse bauchstaendig',
                       '-',
                       '-'], 
        'Besonderheiten':['Augendurchmesser im Verhaeltnis zur Schnauzenlaenge kleiner',
                          '8 Barteln',
                          '6 kurze Barteln, Oberlippe',
                          '4 Barteln, Oberlippe',
                          'kraeftige Fangzaehne, Seitenlinie mehrfach unterbrochen',
                          'Kiemendeckel spitz auslaufend (Dorn)',
                          'Fettflosse, können nicht am gedrungenen Schwanzstiel gehalten werden (im vgl. zum Lachs) '], 
        }
    df = pandas.DataFrame(fish, columns = ['Familie', 'Spezies', 'Name', 'Koerperform', 'Mund', 'Schuppen', 'Farbe', 'Rueckenflosse', 'Schwanzflosse', 'Afterflosse', 'Brust/Bauchflosse','Besonderheiten'], index=['abramis_brama', 'ameiurus_nebulosus', 'cobitis_taenia', 'cyprinus_carpio', 'esox_lucius', 'gymnocephalus_cernua', 'salmo_trutta'])
    return df 

In [12]:
df = pandas.read_csv(path/'fish-information.csv')

In [15]:
df.to_json("fish-information.json")

In [ ]:
def on_data_change(change):
    lbl_pred.value = ''
    #df = create_df()
    img = PILImage.create(btn_upload.data[-1])
    pred,pred_idx,probs = learn_inf.predict(img)
    out_pl.clear_output()
    with out_pl: display(img.to_thumb(128,128))
    out_pl_df.clear_output()
    with out_pl_df: display(df.loc[f'{pred}'])
    lbl_pred.value = f'Prediction: {pred}; Probability: {probs[pred_idx]:.04f}'

In [ ]:
btn_upload.observe(on_data_change, names=['data'])

In [ ]:
display(HBox([VBox([widgets.Label('Select your bear!'), btn_upload, out_pl, lbl_pred]), VBox([widgets.Label('Informationen'), out_pl_df])]))